In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StandardScaler, StringIndexer
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import mean, stddev

spark = SparkSession.builder.appName('ChurnPrediction').getOrCreate()

df = spark.read.option('header', 'true').csv('../data/raw/data.csv', inferSchema=True)

df = df.drop('customerID', 'gender') 
df = df.withColumn("TotalCharges", col("TotalCharges").cast("double"))

df = df.dropDuplicates()
df = df.na.drop()

numerical_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, (IntegerType, DoubleType))]
categorical_cols = [field.name for field in df.schema.fields if field.name not in numerical_cols + ['customerID', 'gender']]

for col_name in numerical_cols:
    col_mean, col_stddev = df.select(mean(col_name).alias('mean'), stddev(col_name).alias('stddev')).first()
    if col_stddev != 0:
        df = df.withColumn(col_name + "_scaled", (col(col_name) - col_mean) / col_stddev)
    else:
        df = df.withColumn(col_name + "_scaled", col(col_name) - col_mean)

for col_name in df.columns:
    if col_name in categorical_cols:
        indexer = StringIndexer(inputCol=col_name, outputCol=col_name + "_indexed")
        df = indexer.fit(df).transform(df)

selected_cols = [col_name for col_name in df.columns if "_scaled" in col_name or "_indexed" in col_name]
df_selected = df.select(*selected_cols)

major_df = df_selected.filter(col('Churn_indexed') == 0)
minor_df = df_selected.filter(col('Churn_indexed') == 1)
major_count = df_selected.count()
minor_count = df_selected.count()
ratio = major_count / minor_count
oversampled_minor_df = minor_df.sample(withReplacement=True, fraction=ratio)
input_data = major_df.unionAll(oversampled_minor_df)

input_data.show(5)
pandas_df = input_data.toPandas()

pandas_df.to_csv('../data/processed/data.csv', index=False)

# spark.stop()

+--------------------+-------------------+---------------------+--------------------+---------------+------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+------------------------+-------------------+-------------------+-----------------------+----------------+------------------------+---------------------+-------------+
|SeniorCitizen_scaled|      tenure_scaled|MonthlyCharges_scaled| TotalCharges_scaled|Partner_indexed|Dependents_indexed|PhoneService_indexed|MultipleLines_indexed|InternetService_indexed|OnlineSecurity_indexed|OnlineBackup_indexed|DeviceProtection_indexed|TechSupport_indexed|StreamingTV_indexed|StreamingMovies_indexed|Contract_indexed|PaperlessBilling_indexed|PaymentMethod_indexed|Churn_indexed|
+--------------------+-------------------+---------------------+--------------------+---------------+------------------+--------------------+---------------------+-----------------------+---------------

In [2]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Selecting the feature columns (scaled and indexed)
feature_cols = [col for col in input_data.columns if col != 'Churn_indexed']

# Assembling the feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

# Setting up the logistic regression model
lr = LogisticRegression(featuresCol='features', labelCol='Churn_indexed')

# Pipeline for the assembler and the logistic model
pipeline = Pipeline(stages=[assembler, lr])

# Splitting data into training and test sets
train_data, test_data = input_data.randomSplit([0.8, 0.2], seed=42)

# Fit the model
model = pipeline.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol='Churn_indexed',
    predictionCol='prediction',
    metricName='accuracy'
)

accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Recall and F1 Score
evaluator_recall_0 = MulticlassClassificationEvaluator(labelCol='Churn_indexed', predictionCol='prediction', metricName='recallByLabel', metricLabel=0)
recall_class_0 = evaluator_recall_0.evaluate(predictions)
print(f"Recall for Class 0: {recall_class_0}")

evaluator_recall_1 = MulticlassClassificationEvaluator(labelCol='Churn_indexed', predictionCol='prediction', metricName='recallByLabel', metricLabel=1)
recall_class_1 = evaluator_recall_1.evaluate(predictions)
print(f"Recall for Class 1: {recall_class_1}")

evaluator_f1 = MulticlassClassificationEvaluator(labelCol='Churn_indexed', predictionCol='prediction', metricName='f1')
f1_score_macro = evaluator_f1.evaluate(predictions)
print(f"F1 Score Macro: {f1_score_macro}")

spark.stop()

Accuracy: 0.8235730170496665
Recall for Class 0: 0.9173387096774194
Recall for Class 1: 0.5630252100840336
F1 Score Macro: 0.8165452457854898


In [4]:
input_data.count()

Py4JJavaError: An error occurred while calling o735.count.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:76)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:53)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:483)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.C